# Prueba Jr Data Analyst TalentPitch

## 2. Recorrer los links del dataset en paralelo

In [1]:
# Importar librerias necesarias 
import pandas as pd
from bs4 import BeautifulSoup
import requests
from lxml import etree
import concurrent.futures

In [2]:
# Leer el dataset que ya hemos guardado anteriormente para utilizar los link y conseguir la información 
# de cargo, palabras clave y descripcion de cada uno de los link 
df =pd.read_csv('DatasetLinks.csv')

In [23]:
## Pasar a lista todas las url
urls=df['Urls'].to_list()
contenidos=[]
titulos=[]
palabrasclaves=[]

# Funcion para hacer scraping con request
def scrape_webpage(url):
    response = requests.get(url)
    return response.content


# Obtener las variables buscadas (cargo, palabras clave y descripcion)
def ObtenerTitCont(url):
    soup = BeautifulSoup(scrape_webpage(url), "html.parser")
    dom = etree.HTML(str(soup))
    
    
    contenido=dom.xpath("//div[@class='container js-offer-detail-container']/div[3]/div[1]/div[1]/div/p[1]/span")[0].text
    contenidos.append(contenido)

    palabraclave=dom.xpath("//div[@class='container js-offer-detail-container']/div[3]/div[1]/div[1]/div/div[2]/div/ul/li/span")[0].text
    palabrasclaves.append(palabraclave)

    h1 = soup.find("h1").text
    titulos.append(h1)
    return

## La libreria concurrent.futures.ThreadPoolExecutor nos permite ejecutar multiples tareas al tiempo dandole
## los parametros de lo que hay que hacer y cuales son los caminos(urls)
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(ObtenerTitCont, urls)

## Limpiar y transformar la data  a un csv
titulos2 = [element.replace("\n", "").replace("\r", "").replace("\\", "") for element in titulos]
contenidos2 = [element.replace("\r\n            ", "").replace("\r", "").replace("\\", "").replace("\n", "") for element in contenidos]
palabrasclaves2=[element.replace("\r\n            ", "").replace("\r", "").replace("\\", "").replace("\n", "") for element in palabrasclaves]
df1=pd.DataFrame({"Titulos":titulos2,"contenidos":contenidos2,"Palabras claves":palabrasclaves2}) 




In [24]:
df1.to_csv('EmpleosTitulosyDescripcion.csv',index=False)

In [19]:
df1

,Titulos,contenidos,Palabras claves
0,Auxiliar compras,Importante empresa requiere Auxiliar de compra...,Auxiliar compr...
1,Auxiliar mecánico,"Importante multinacional del sector eléctrico,...",mecanico ...
2,Auxiliar administrativa cartagena,Importante empresa en la ciudad de Cartagena r...,auxiliar ...
3,Auxiliar punto de venta,Requerimos personal dinámico y con actitud de ...,auxiliar ...
4,Enfermero jefe -,Para los diferentes servivios de Institución d...,Enfermero Jefe...
...,...,...,...
9986,Oficial de obra zipaquira,Nos encontramos en la búsqueda de Oficial de O...,oficial de obr...
9987,Analista de desarrollo informatico,Importante compañía del sector financiero requ...,programación d...
9988,Auxiliar de reclutamiento y selección,En KOAJ - Permoda se abre una nueva oportunida...,talento humano...
9989,Optómetra,"Importante multinacional de consumo masivo, re...",OPTOMETRA ...
